<a href="https://colab.research.google.com/github/BradleyMerrillT/Indiana-Expungement/blob/main/EXIndianaMisdClean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EX, the expungement app for Indiana ex-offenders

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#imports
import pandas as pd
import re
import numpy as np
import math
import zipfile
import os
import pickle

In [3]:
df = pd.read_pickle("/content/drive/MyDrive/Expungement/dataframe.pkl")

#Ind. Code 35-38-9-2 (Misdemeanors)

Current as of January 02, 2024 | Updated by FindLaw Staff

Sec. 2. (a) Except as provided in subsection (b) and section 8.5 of this chapter, this section applies only to a person convicted of a misdemeanor, including a Class D felony (for a crime committed before July 1, 2014) or a Level 6 felony (for a crime committed after June 30, 2014) reduced to a misdemeanor.

(b) This section does not apply to the following:

(1) A person convicted of two (2) or more felony offenses that:

(A) involved the unlawful use of a deadly weapon; and

(B) were not committed as part of the same episode of criminal conduct.

(2) A sex or violent offender (as defined in IC 11-8-8-5).

(c) Not earlier than five (5) years after the date of conviction (unless the prosecuting attorney consents in writing to an earlier period) for the misdemeanor or the felony reduced to a misdemeanor pursuant to IC 35-38-1-1.5 or IC 35-50-2-7, the person convicted of the misdemeanor or the felony reduced to a misdemeanor may petition a court to expunge all conviction records, including records contained in:

(1) a court's files;

(2) the files of the department of correction;

(3) the files of the bureau of motor vehicles; and

(4) the files of any other person who provided treatment or services to the petitioning person under a court order;

that relate to the person's misdemeanor conviction, including records of a collateral action.

(d) A person who files a petition to expunge conviction records, including any records relating to the conviction and any records concerning a collateral action, shall file the petition in a circuit or superior court in the county of conviction.

(e) If the court finds by a preponderance of the evidence that:

(1) the period required by this section has elapsed;

(2) no charges are pending against the person;

(3) the person has paid all fines, fees, and court costs, and satisfied any restitution obligation placed on the person as part of the sentence; and

(4) the person has not been convicted of a crime within the previous five (5) years (or within a shorter period agreed to by the prosecuting attorney if the prosecuting attorney has consented to a shorter period under subsection (c));

the court shall order the conviction records described in subsection (c), including any records relating to the conviction and any records concerning a collateral action, expunged in accordance with section 6 of this chapter.

###Now Section by section

#Subsection (a)

(a) Except as provided in subsection (b) and section 8.5 of this chapter, this section applies only to a person convicted of a misdemeanor, including a Class D felony (for a crime committed before July 1, 2014) or a Level 6 felony (for a crime committed after June 30, 2014) reduced to a misdemeanor.

For felony reduce rules, see IN Code § 35-50-2-7 (2024)

Infractions are covered in IC 34-28-5-15

For reference,
IN Code § 35-38-9-8.5 (2024)

Sec. 8.5. (a) This section applies only to a person seeking to expunge an Indiana offense punishable by an indeterminate sentence under a law other than IC 35-50.

(b) If the offense for which the person was convicted is a misdemeanor at the time the person files the petition for expungement, the person may file the petition for expungement under section 2 of this chapter.

(c) If the offense for which the person was convicted:

(1) is a Level 6 felony at the time the person files the petition for expungement; and

(2) is not substantially similar to an offense described in section 3(b) of this chapter;

the person may file the petition under section 3 of this chapter.

(d) If:

(1) the person to whom this chapter applies may not seek expungement under section 3 of this chapter; and

(2) the offense the person seeks to expunge is not substantially similar to an offense described in section 4(b) of this chapter;

the person may file the petition under section 4 of this chapter.

(e) If the offense for which the person was convicted:

(1) is a felony at the time of filing the petition, including a felony described in section 5(a) of this chapter; and

(2) is not substantially similar to an offense described in section 5(b) of this chapter;

the person may file the petition under section 5 of this chapter.

As added by P.L.142-2015, SEC.9.

In [4]:
df['DisposedDegree'] = df['DisposedDegree'].fillna(False)

In [5]:
df['DisposedDegree'].unique()

array([False, 'Misdemeanor Class B', 'Misdemeanor Class C',
       'Misdemeanor Class A', 'Degree Code', 'Infraction Class C',
       'Felony B', 'Felony A', 'Felony D', 'Felony C', 'No State Code',
       'Murder', 'Infraction Class A', 'Infraction Class D',
       'Infraction Class B', 'Misdemeanor Class D', 'Felony 6',
       'Felony 5', 'Felony 2', 'Felony 4', 'Felony 3', 'Felony 1',
       'Misdemeanor'], dtype=object)

In [6]:
#Filter for misdemeanor convictions
df_misd = df[df['DisposedDegree'].str.contains("Misdemeanor", na=False)]

In [7]:
len(df_misd)

3421079

#Subsection (b)

(b) This section does not apply to the following:

(1) A person convicted of two (2) or more felony offenses that:

(A) involved the unlawful use of a deadly weapon; and

(B) were not committed as part of the same episode of criminal conduct.

(2) A sex or violent offender (as defined in IC 11-8-8-5).

Going forward, can we ask courts to check a box on conviction regarding whether an offense meets these criteria or not?

##Subsection (b)(1)

In [8]:
#Filter for Felonies
df_fel = df[df['DisposedDegree'].str.contains("Felony", na=False)]

In [9]:
len(df_fel)

1551202

Filter for felonies by a partial list of those statutory categories of felonies that don't involve weapons

In [10]:
#filter out offenses that clearly do not meet the exclusion standard
Ok_offenses = ['35-43-4', '35-48-4', '35-45-1',  '9-30-5', '16-42-19', '35-43-2-1.5', '35-43-5']
df_not_ok = df_fel[~df_fel['DisposedStatute'].str.contains('|'.join(Ok_offenses), na=False) & (df['DisposedStatute'].str.len() > 14)]

<ipython-input-10-e5783a55b727>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_not_ok = df_fel[~df_fel['DisposedStatute'].str.contains('|'.join(Ok_offenses), na=False) & (df['DisposedStatute'].str.len() > 14)]


In [11]:
len(df_not_ok)

342803

With the felony list slimmed down, who are the repeat offenders

In [12]:
#Who has 2 or more felonies?--need to add except on same date
df_repeat = df_not_ok.groupby('PartyID').filter(lambda x: len(x) >= 2)

In [13]:
len(df_repeat)

222591

In [14]:
repeat = df_repeat['PartyID'].unique()

In [15]:
len(repeat)

72587

##Subsection (b)(2)

Indiana Code Title 11. Corrections § 11-8-8-5
Current as of January 02, 2024

Sec. 5. (a) Except as provided in section 22 of this chapter, as used in this chapter, “sex or violent offender” means a person convicted of any of the following offenses:

(1) Rape (IC 35-42-4-1).

(2) Criminal deviate conduct (IC 35-42-4-2) (before its repeal).

(3) Child molesting (IC 35-42-4-3).

(4) Child exploitation (IC 35-42-4-4(b) or IC 35-42-4-4(c)).

(5) Vicarious sexual gratification (including performing sexual conduct in the presence of a minor) (IC 35-42-4-5).

(6) Child solicitation (IC 35-42-4-6).

(7) Child seduction (IC 35-42-4-7).

(8) Sexual misconduct with a minor (IC 35-42-4-9) as a Class A, Class B, or Class C felony (for a crime committed before July 1, 2014) or a Level 1, Level 2, Level 4, or Level 5 felony (for a crime committed after June 30, 2014), unless:

(A) the person is convicted of sexual misconduct with a minor as a Class C felony (for a crime committed before July 1, 2014) or a Level 5 felony (for a crime committed after June 30, 2014);

(B) the person is not more than:

(i) four (4) years older than the victim if the offense was committed after June 30, 2007; or

(ii) five (5) years older than the victim if the offense was committed before July 1, 2007; and

(C) the sentencing court finds that the person should not be required to register as a sex offender.

(9) Incest (IC 35-46-1-3).

(10) Sexual battery (IC 35-42-4-8).

(11) Kidnapping (IC 35-42-3-2), if the victim is less than eighteen (18) years of age, and the person who kidnapped the victim is not the victim's parent or guardian.

(12) Criminal confinement (IC 35-42-3-3), if the victim is less than eighteen (18) years of age, and the person who confined or removed the victim is not the victim's parent or guardian.

(13) Possession of child pornography (IC 35-42-4-4(d) or IC 35-42-4-4(e)).

(14) Promoting prostitution (IC 35-45-4-4) as a Class B felony (for a crime committed before July 1, 2014) or a Level 4 felony (for a crime committed after June 30, 2014).

(15) Promotion of human sexual trafficking under IC 35-42-3.5-1.1.

(16) Promotion of child sexual trafficking under IC 35-42-3.5-1.2(a).

(17) Promotion of sexual trafficking of a younger child (IC 35-42-3.5-1.2(c)).

(18) Child sexual trafficking (IC 35-42-3.5-1.3).

(19) Human trafficking under IC 35-42-3.5-1.4 if the victim is less than eighteen (18) years of age.

(20) Murder (IC 35-42-1-1).

(21) Voluntary manslaughter (IC 35-42-1-3).

(22) Sexual misconduct by a service provider with a detained or supervised child (IC 35-44.1-3-10(c)).

(b) The term includes:

(1) a person who is required to register as a sex or violent offender in any jurisdiction; and

(2) a child who has committed a delinquent act, or a person prosecuted under IC 31-30-1-4(d) for an offense described in subsection (a) committed when the person was less than eighteen (18) years of age, but who was at least twenty-one (21) years of age when the charge was filed, and who:

(A) is at least fourteen (14) years of age;

(B) is on probation, is on parole, is discharged from a facility by the department of correction, is discharged from a secure private facility (as defined in IC 31-9-2-115), or is discharged from a juvenile detention facility as a result of an adjudication as a delinquent child for an act that would be an offense described in subsection (a) if committed by an adult; and

(C) is found by a court by clear and convincing evidence to be likely to repeat an act that would be an offense described in subsection (a) if committed by an adult.

(c) In making a determination under subsection (b)(2)(C), the court shall consider expert testimony concerning whether a child is likely to repeat an act that would be an offense described in subsection (a) if committed by an adult.

(d) A person ordered to register under subsection (b)(2) may petition the court to reconsider the order at any time after completing court ordered sex offender treatment. The court shall consider expert testimony concerning whether a child or person is likely to repeat an offense described in subsection (a) or an act that would be an offense described in subsection (a) if committed by an adult.

In [16]:
# here we want to filter the misdemenor list by the list of sex crimes to make sure none of the included misdemenor can be forbidden
sex_offenses = ['35-42-4-1', '35-42-4-2', '35-42-4-3',  '35-42-4-4', '35-42-4-5', '35-42-4-6', '35-42-4-7', '35-42-4-9', '35-46-1-3', '35-42-4-8', '35-42-3-2', '35-42-3-3', '35-42-4-4', '35-45-4-4', '35-42-3.5-1.1', '35-42-3.5-1.2', '35-42-3.5-1.3', '35-42-3.5-1.4', '35-42-1-3', '35-44.1-3-10', '31-30-1-4']
df_misd_no_sex = df_misd[~df_misd['DisposedStatute'].str.contains('|'.join(sex_offenses), na=False)]

In [17]:
len(df_misd_no_sex)

3420179

In [18]:
df_misd_sex = df_misd[df_misd['DisposedStatute'].str.contains('|'.join(sex_offenses), na=False)]

In [19]:
len(df_misd_sex)

900

We need to get access to more data to inplement "a person who is required to register as a sex or violent offender in any jurisdiction"

#Filter subsection (a) by (b)(1)

In [20]:
df_b = df_misd_no_sex[~df_misd_no_sex['PartyID'].isin(repeat)]

In [21]:
len(df_b)

3071712

#Subsection (c)

(c) Not earlier than five (5) years after the date of conviction (unless the prosecuting attorney consents in writing to an earlier period) for the misdemeanor or the felony reduced to a misdemeanor pursuant to IC 35-38-1-1.5 or IC 35-50-2-7, the person convicted of the misdemeanor or the felony reduced to a misdemeanor may petition a court to expunge all conviction records,

In [22]:
filter_date = '2021-12-01'
df_c = df_b[df_b['DispositionDate'] < filter_date]

In [23]:
len(df_c)

2466485

#Subsection (d) skip as there are no criteria there

#Subsection (e)

(e) If the court finds by a preponderance of the evidence that:

(1) the period required by this section has elapsed;

(2) no charges are pending against the person;

(3) the person has paid all fines, fees, and court costs, and satisfied any restitution obligation placed on the person as part of the sentence; and

(4) the person has not been convicted of a crime within the previous five (5) years (or within a shorter period agreed to by the prosecuting attorney if the prosecuting attorney has consented to a shorter period under subsection (c));

subections (1) and (4) have been addressed above

In [24]:
#(2) no charges are pending against this person
#charges database where there is no disposition
df_pending = df[df['Disposition'].isnull()]
pending = df_pending['PartyID'].unique()

In [25]:
len(pending)

377758

In [26]:
#filter out those who have pending case
df_e_1 = df_c[~df_c['PartyID'].isin(pending)]

In [27]:
len(df_e_1)

2117662

In [28]:
#(3) the person paid all fines etc
df_e = df_e_1[df_e_1['FeeBalance'] <= 0]

In [29]:
len(df_e)

1299055

In [30]:
df_e.head()

,CaseID,PartyID,ChargeID,CountNumber,OffenseDate,ChargedStatute,ChargedOffense,ChargedDegree,ConvictedStatute,ConvictedOffense,ConvictedDegree,ConvictedDisposition,ConvictedDate,DisposedStatute,DisposedOffense,DisposedDegree,Disposition,DispositionDate,FeeBalance
1,244739,862741,75307,1,12-31-2006,7.1-5-1-3,PUBLIC INTOXICATION,Misdemeanor Class B,7.1-5-1-3,PUBLIC INTOXICATION,Misdemeanor Class B,Finding of Guilty,03-14-2007,7.1-5-1-3,PUBLIC INTOXICATION,Misdemeanor Class B,Finding of Guilty,2007-03-14,0.0
2,244739,862741,75308,2,12-31-2006,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,Finding of Guilty,03-14-2007,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,Finding of Guilty,2007-03-14,0.0
3,244740,862743,75309,1,12-28-2006,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,Finding of Guilty,09-12-2007,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,Finding of Guilty,2007-09-12,0.0
8,244744,862747,75314,2,01-02-2007,9-24-19-2,DRIVING WHILE SUSPENDED,Misdemeanor Class A,9-24-19-2,DRIVING WHILE SUSPENDED,Misdemeanor Class A,Finding of Guilty,03-06-2007,9-24-19-2,DRIVING WHILE SUSPENDED,Misdemeanor Class A,Finding of Guilty,2007-03-06,-5.0
9,244745,862749,75315,1,12-28-2006,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,Finding of Guilty,01-02-2007,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,Finding of Guilty,2007-01-02,0.0
